In [ ]:
# Mount the Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Setting System to Import Packages
import sys
sys.path.append('/content/drive/MyDrive/Earthquake/WorkingScripts')

In [ ]:
# Install Required Packages
import environment_setting as es # Replace my_module with the actual name of your module
parent_dir = '/content/drive/MyDrive/Earthquake/'
es.setting_env(parent_dir)

In [ ]:
# Running o1
import o1_getshakemap as getshakemap
EVENTID = "nc72282711"  # Example event ID
feed_url = getshakemap.FEEDURL.format(EVENTID)
jdict = getshakemap.fetch_earthquake_data(feed_url=feed_url)

event = getshakemap.retrieve_event_data(jdict)
getshakemap.download_and_extract_shakemap(event)